In [1]:
import scipy.stats as scs
from coins import BayesCoin
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.model_selection import train_test_split
import spacy
import nltk
from spacy import displacy
from sklearn import metrics
from itertools import combinations
from nltk.corpus import stopwords
set(stopwords.words('english'))
import unicodedata
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.util import ngrams

%load_ext autoreload
%autoreload 2

# Part 1

In [177]:
coin = BayesCoin()

In [178]:
coin.choose_coin()

In [179]:
coin.coins

[0.3, 0.45, 0.75]

In [180]:
coin.data

{0.3: 0.3333333333333333, 0.45: 0.3333333333333333, 0.75: 0.3333333333333333}

In [181]:
coin.flip_coin()

1

In [182]:
coin.coin

0.45

In [183]:
coin.update_priors(coin.flip_coin())

before update: {0.3: 0.3333333333333333, 0.45: 0.3333333333333333, 0.75: 0.3333333333333333}
coin: 0.45 flip: 0
denominator: [0.2333333333333333, 0.18333333333333335, 0.08333333333333333]
data: {0.3: 0.4666666666666667, 0.45: 0.36666666666666675, 0.75: 0.16666666666666669}
sum of priors: 1.0000000000000002
sum of denominator: 0.49999999999999994
--------------------------------------------------


In [184]:
coin.data

{0.3: 0.4666666666666667, 0.45: 0.36666666666666675, 0.75: 0.16666666666666669}

In [190]:
for i in range(10):
    flip = coin.flip_coin()
    coin.update_priors(flip)

before update: {0.3: 0.6490063467285541, 0.45: 0.35092427227837275, 0.75: 6.938099307317753e-05}
coin: 0.45 flip: 1
denominator: [0.1947019040185662, 0.15791592252526773, 5.2035744804883145e-05]
data: {0.3: 0.552079791437394, 0.45: 0.4477726605286821, 0.75: 0.00014754803392384873}
sum of priors: 0.9999999999999999
sum of denominator: 0.35266986228863884
--------------------------------------------------
before update: {0.3: 0.552079791437394, 0.45: 0.4477726605286821, 0.75: 0.00014754803392384873}
coin: 0.45 flip: 1
denominator: [0.1656239374312182, 0.20149769723790695, 0.00011066102544288656]
data: {0.3: 0.45100591471118817, 0.45: 0.5486927473434832, 0.75: 0.000301337945328547}
sum of priors: 0.9999999999999998
sum of denominator: 0.36723229569456806
--------------------------------------------------
before update: {0.3: 0.45100591471118817, 0.45: 0.5486927473434832, 0.75: 0.000301337945328547}
coin: 0.45 flip: 1
denominator: [0.13530177441335645, 0.24691173630456742, 0.00022600345899

In [188]:
coin.coin

0.45

# Part II

#### Q1 - Answer: Assumption that data/events are independent

#### Q2 - Answer: Bernoulli is when all values are either 0 or 1, Multinomial is when the data is being counted, Gaussian is for conditional data

#### Q3 - Answer: No, independence is required for reliable output

# Part III

In [194]:
df = pd.read_csv('grad.csv')
df.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [195]:
y = df.pop('admit')
X = df

In [204]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=21)
scalar = StandardScaler().fit(X_train)
X_train = scalar.transform(X_train)
X_test = scalar.transform(X_test)

C:\Users\e0wrvbu\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\e0wrvbu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\e0wrvbu\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


In [205]:
LogisticRegression().fit(X_train, y_train).score(X_test, y_test)

C:\Users\e0wrvbu\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7651515151515151

In [206]:
RandomForestClassifier().fit(X_train, y_train).score(X_test, y_test)

C:\Users\e0wrvbu\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6818181818181818

In [207]:
GaussianNB().fit(X_train, y_train).score(X_test, y_test)
# Naive-Bayes Gaussian model performed best

0.7954545454545454

# Part IV

In [2]:
nlp = spacy.load("en_core_web_lg")

In [3]:
tweets = pd.read_csv('Tweets.csv')
tweets.head(1)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)


In [246]:
y = tweets.pop('airline_sentiment')
X = tweets.text

In [247]:
# turn X into a corpus of documents from a series
corpus = ' '.join(X)

In [226]:
# normalize any unicode text within the corpus, replacing all compatibility characters with their equivalents (compatibility decomposition)
normalize = (unicodedata.normalize('NFKD', corpus).encode('ASCII', 'ignore').decode('utf-8'))

In [249]:
# tokenize
tokens = list(map(lambda s: word_tokenize(s.lower()), sent_tokenize(corpus)))

21482

In [250]:
# remove stop words
sw = stopwords.words('english')
pt = string.punctuation
filtered = [list(filter(lambda token: token not in sw and token not in pt, row)) for row in tokens]

In [251]:
stemmer_snowball = SnowballStemmer('english')
tokens_stemsnowball = [list(map(stemmer_snowball.stem, row)) for row in filtered]

In [252]:
documents = [row + list(map(lambda ng: '-'.join(ng), ngrams(row, 2))) for row in tokens_stemsnowball]

In [253]:
# creates a set of all words in the corpus, creates a lookup from a word to its index/position, initializes an empty numpy matrix to be used to vectorize the documents
vocabulary = set()
[[vocabulary.add(token) for token in row] for row in documents]
vocabulary_lookup = {word: i for i,word in enumerate(vocabulary)}
matrix = np.zeros((len(documents), len(vocabulary)))

In [254]:
# count words in each document
for doc_id, document in enumerate(documents):
    for word in document:
        word_id = vocabulary_lookup[word]
        matrix[doc_id][word_id] += 1

In [257]:
# calculate term frequency
tf = matrix/np.sum(matrix, axis=1).reshape(14640, 1)
tf[0]

ValueError: cannot reshape array of size 21482 into shape (14640,1)

In [236]:
tokens[0]

['@', 'virginamerica', 'what', '@', 'dhepburn', 'said', '.']

In [237]:
len(X)

14640